# Queries with and without Azure OpenAI

Now that we have our Search Engine loaded **from two different data sources in two diferent indexes**, we are going to try some example queries and then use Azure OpenAI service to see if we can get even better results.

The idea is that a user can ask a question about Computer Science (first datasource/index) or about Covid (second datasource/index), and the engine will respond accordingly.
This **Multi-Index** demo, mimics the scenario where a company loads multiple type of documents of different types and about completly different topics and the search engine must respond with the most relevant results.

## Set up variables

In [1]:
import os
import urllib
import requests
from IPython.display import display, HTML
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

from app.embeddings import OpenAIEmbeddings
from app.prompts import STUFF_PROMPT, REFINE_PROMPT, REFINE_QUESTION_PROMPT
from app.credentials import (
    DATASOURCE_CONNECTION_STRING,
    AZURE_SEARCH_API_VERSION,
    AZURE_SEARCH_ENDPOINT,
    AZURE_SEARCH_KEY,
    COG_SERVICES_NAME,
    COG_SERVICES_KEY,
    AZURE_OPENAI_ENDPOINT,
    AZURE_OPENAI_KEY,
    AZURE_OPENAI_API_VERSION
)

In [2]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': AZURE_SEARCH_KEY}
params = {'api-version': AZURE_SEARCH_API_VERSION}

## Multi-Index Search queries

In [3]:
# Index that we are going to query (from Notebook 01 and 02)
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
indexes = [index1_name, index2_name]

Try questions that you think might be answered or addressed in computer science papers in 2020-2021 or that can be addressed by medical publications about COVID in 2020. Try comparing the results with the open version of ChatGPT.<br>
The idea is that the answers using Azure OpenAI only looks at the information contained on these publications.

**Example Questions you can ask**:
- What is CLP?
- How Markov chains work?
- What are some examples of reinforcement learning?
- What are the main risk factors for Covid-19?
- What medicine reduces inflamation in the lungs?
- Why Covid doesn't affect kids that much compared to adults?

In [4]:
QUESTION = "What is CLP?" # This questions is interesting since CLP means something in Computer science and means something different in medical field
# QUESTION = "What are decission trees?" 

### Search on both indexes individually and aggragate results

In [5]:
agg_search_results = []

for index in indexes:
    url = AZURE_SEARCH_ENDPOINT + '/indexes/'+ index + '/docs'
    url += '?api-version={}'.format(AZURE_SEARCH_API_VERSION)
    url += '&search={}'.format(QUESTION)
    url += '&select=*'
    url += '&$top=5'  # You can change this to anything you need/want
    url += '&queryLanguage=en-us'
    url += '&queryType=semantic'
    url += '&semanticConfiguration=my-semantic-config'
    url += '&$count=true'
    url += '&speller=lexicon'
    url += '&answers=extractive|count-3'
    url += '&captions=extractive|highlight-false'

    resp = requests.get(url, headers=headers)
    print(url)
    print(resp.status_code)

    search_results = resp.json()
    agg_search_results.append(search_results)
    print("Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))

https://azure-cog-search-cstevuxaqrxcm.search.windows.net/indexes/cogsrch-index-files/docs?api-version=2021-04-30-Preview&search=What is CLP?&select=*&$top=5&queryLanguage=en-us&queryType=semantic&semanticConfiguration=my-semantic-config&$count=true&speller=lexicon&answers=extractive|count-3&captions=extractive|highlight-false
200
Results Found: 880, Results Returned: 5
https://azure-cog-search-cstevuxaqrxcm.search.windows.net/indexes/cogsrch-index-csv/docs?api-version=2021-04-30-Preview&search=What is CLP?&select=*&$top=5&queryLanguage=en-us&queryType=semantic&semanticConfiguration=my-semantic-config&$count=true&speller=lexicon&answers=extractive|count-3&captions=extractive|highlight-false
200
Results Found: 21475, Results Returned: 5


### Display the top results (from both searches) based on the score

In [8]:
display(HTML('<h4>Top Answers</h4>'))

for search_results in agg_search_results:
    for result in search_results['@search.answers']:
        if result['score'] > 0.5: # Show answers that are at least 50% of the max possible score=1
            display(HTML('<h5>' + 'Answer - score: ' + str(result['score']) + '</h5>'))
            display(HTML(result['text']))
            
print("\n\n")
display(HTML('<h4>Top Results</h4>'))

file_content = dict()

for search_results in agg_search_results:
    for result in search_results['value']:
        if result['@search.rerankerScore'] > 0.4: # Show results that are at least 10% of the max possible score=4
            display(HTML('<h5>' + result['title'] + '&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;score: '+ str(result['@search.rerankerScore']) + '</h5>'))
            display(HTML(result['@search.captions'][0]['text']))
            file_content[result['id']]={
                                    "title": result['title'],
                                    "content": result['pages'],  # result['chunks'] can be used here as well
                                    "caption": result['@search.captions'][0]['text'],
                                    "score": result['@search.rerankerScore'],
                                    "location": result['metadata_storage_path']                  
                                }

## Comments on Query results

As seen above the semantic search feature of Azure Cognitive Search service is good. It gives us some answers and also the top results with the corresponding file and the paragraph where the answers is possible located.

Let's see if we can make this better with Azure OpenAI

# Using Azure OpenAI

Of course we want OpenAI to give a better answer, so we instead of sending these results, we send the content of the documents of the search result articles to OpenAI and lets GPT model give the answer.

The problem is that the content of the search result files is or can be very lengthy, more than the allowed tokens allowed by the GPT Azure OpenAI models. So what we need to do is to split in chunks, vectorize and do a vector semantic search. 

Notice that **the documents chunks are already done in Azure Search**. file_content dictionary (created in the cell above) contains the pages (chunks) of each document. So we dont really need to chunk them again, each doc page for sure will fit on the max tokens limit of the completions LLM and of the embedding LLM.

We will use a genius library call LangChain that wraps a lot of boiler plate code.

In [9]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION

In [10]:
# In Azure OpenAI create a deployment for the model "text-embedding-ada-002"
# and VERY IMPORTANT name the deployment the same: "text-embedding-ada-002"
embeddings = OpenAIEmbeddings()

In [12]:
docs = []
for key,value in file_content.items():
    for page in value["content"]:
        docs.append(Document(page_content=page, metadata={"source": value["location"]}))

In [13]:
%%time
if(len(docs)>1):
    db = FAISS.from_documents(docs, embeddings)
else:
    print("No results Found")

CPU times: user 330 ms, sys: 35 ms, total: 365 ms
Wall time: 29.2 s


In [14]:
docs_db = db.similarity_search(QUESTION)

At this point we already have the most similar chuncks (in order of relevance given by the in-memory vector cosine similarity search) in docs_db

### Now we use GPT-3.5(Turbo) using map-reduce chain in order to stay within the limits of the allow model's token count

for more information on the different types of prompts for these chains please see here:

https://github.com/hwchase17/langchain/tree/master/langchain/chains/question_answering

In [15]:
# Make sure you have the deployment named "gpt-35-turbo" for the model "gpt-35-turbo (0301)"
llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0.3, max_tokens=500)
chain = load_qa_with_sources_chain(llm, chain_type="map_reduce", return_intermediate_steps=True)

In [19]:
%%time
response = chain({"input_documents": docs_db, "question": QUESTION}, return_only_outputs=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (1905 > 1024). Running this sequence through the model will result in indexing errors


CPU times: user 223 ms, sys: 4.66 ms, total: 228 ms
Wall time: 18.5 s


In [20]:
answer = response['output_text']

display(HTML('<h4>Azure OpenAI ChatGPT Answer:</h4>'))
print(answer.split("SOURCES:")[0])
print("Sources:")
print(answer.split("SOURCES:")[1].replace(" ","").split(","))

CLP stands for Constraint Logic Programming, which is a powerful extension of conventional logic programming involving the incorporation of constraint languages and constraint solving methods into logic programming languages. It can also refer to Consultation-Liaison psychiatry services and finite domain CLP, which is widely accepted as an excellent tool for CSP solving. 

Sources:
['https://demodatasetsp.blob.core.windows.net/arxivcs/0008/0008036v1.pdf', 'https://demodatasetsp.blob.core.windows.net/litcovid/train.csv', 'https://demodatasetsp.blob.core.windows.net/arxivcs/0011/0011030v1.pdf']


In [21]:
# Uncomment if you want to inspect the results from each top similar chunk
response['intermediate_steps']

['Constraint logic programming (CLP) is a powerful extension of conventional logic programming involving the incorporation of constraint languages and constraint solving methods into logic programming languages. The name CLP was first introduced by Jaffar and Lassez (1986) for a general framework of a logic programming language that is parametrized with respect to constraint language and a domain of computation.',
 'Consultation-Liaison psychiatry (CLP) services.',
 'CLP stands for Constraint Logic Programming. It is a scheme that embeds arbitrary logical languages into constraint logic programs, defining grammars as constraint logic programs, i.e., as sets of axiomatic interpreted definite clauses. The prediction problem is in this setting as follows: Given a program P (encoding a grammar) and a definite goal G (encoding the string/logical form we want to parse/generate from), we ask if we can infer an answer ϕ of G (which is a satisfiable constraint encoding an analysis) proving the 

# Summary
##### This answer is way better than taking just the result from Azure Cognitive Search. So the summary is:
- Azure Cognitive Search give us the top results (context)
- Azure OpenAI takes these results and understand the content and uses it as context to give the best answer
- Best of two worlds!